In [58]:
import pandas as pd
import numpy as np

Two datasets - 

CDP based GHG emissions data - https://data.cdp.net/Companies/2013-Global-500-Emissions-and-Response-Status/marp-zazk
S&P500 - Balance Sheet and Income Data - https://simfin.com/data/bulk

For faster processing, i combined the three data sets in excel, however the process can be easily replicated in python using merge function.Broadly, the steps for pre-processing are- 

1. Filtering for US based companies and any blanks in Scope 1/Scope 2 data. 
2. Introduce a new variable ticker in data downloaded from SimFin which is Ticker symbol + "US"
3. Combine both these data, with merge on ticker name. 


In [59]:
#loading the dataset
path = r"C:\Users\Gaurav\Downloads\sp_data.xlsx"

data = pd.read_excel(path)
data = data.dropna(axis=0)
data.drop(["Unnamed: 15","Unnamed: 16"],axis=1,inplace=True)
data.head()

,Company Name,Account Number,Ticker Symbol,ISIN,Disclosure Score,Performance Band,Permission,Response Status,Scope 1 (metric tonnes CO2e),Scope 2 (metric tonnes CO2e),Ticker,Reporting,Total Assets,Total Equity,Total Liabilities,Revenue,Gross Profit,Research & Development,Operating Income (Loss),Net Income (Common)
0,CSX Corporation,4120,CSX US,US1264081035,95.0,A,Public,AQ*,5268905.0,300170.0,CSX US,2016.0,35414.0,11694.0,23720.0,11069.0,4690.0,0.0,3389.0,1714.0
2,3M Company,285,MMM US,US88579Y1010,70.0,D,Public,AQ*,4540000.0,2230000.0,MMM US,2016.0,32906.0,10343.0,22563.0,30109.0,15069.0,-1735.0,7112.0,5050.0
5,"The Home Depot, Inc.",8526,HD US,US4370761029,99.0,A-,Public,AQ*,250224.0,2529646.0,HD US,2015.0,39946.0,9322.0,30624.0,88519.0,30265.0,0.0,11774.0,7009.0
6,Las Vegas Sands Corporation,10494,LVS US,US5178341070,98.0,A-,Public,AQ*,225157.0,821527.0,LVS US,2016.0,20469.0,7496.0,12973.0,11271.0,5888.0,-9.0,2581.0,1679.0
16,FedEx Corporation,6287,FDX US,US31428X1063,80.0,B,Public,AQ*,14602697.0,959109.0,FDX US,2016.0,45959.0,13784.0,32175.0,50365.0,14457.0,0.0,4575.0,1820.0


CDP Data points 

Performance Band - Corresponds to a particular Scoring Level (Disclosure, Awareness, Management, Leadership) which demonstrates the company’s level of environmental stewardship and your actions and approaches in managing climate change. 



Ref - https://www.esg.adec-innovations.com/assets/img/whitepaper/pdf/4-steps-to-improving-your-cdp-performance-whitepaper-12072017_2.pdf



In [60]:
# We define a combined Emission = Scope 1 + Scope 2

data["Total Emissions"] = data["Scope 1 (metric tonnes CO2e)"]+data["Scope 2 (metric tonnes CO2e)"]
data["Emissions / Assets"] = data["Total Emissions"]/data["Total Assets"]


data.head()

,Company Name,Account Number,Ticker Symbol,ISIN,Disclosure Score,Performance Band,Permission,Response Status,Scope 1 (metric tonnes CO2e),Scope 2 (metric tonnes CO2e),...,Total Assets,Total Equity,Total Liabilities,Revenue,Gross Profit,Research & Development,Operating Income (Loss),Net Income (Common),Total Emissions,Emissions / Assets
0,CSX Corporation,4120,CSX US,US1264081035,95.0,A,Public,AQ*,5268905.0,300170.0,...,35414.0,11694.0,23720.0,11069.0,4690.0,0.0,3389.0,1714.0,5569075.0,157.256311
2,3M Company,285,MMM US,US88579Y1010,70.0,D,Public,AQ*,4540000.0,2230000.0,...,32906.0,10343.0,22563.0,30109.0,15069.0,-1735.0,7112.0,5050.0,6770000.0,205.737555
5,"The Home Depot, Inc.",8526,HD US,US4370761029,99.0,A-,Public,AQ*,250224.0,2529646.0,...,39946.0,9322.0,30624.0,88519.0,30265.0,0.0,11774.0,7009.0,2779870.0,69.590697
6,Las Vegas Sands Corporation,10494,LVS US,US5178341070,98.0,A-,Public,AQ*,225157.0,821527.0,...,20469.0,7496.0,12973.0,11271.0,5888.0,-9.0,2581.0,1679.0,1046684.0,51.135082
16,FedEx Corporation,6287,FDX US,US31428X1063,80.0,B,Public,AQ*,14602697.0,959109.0,...,45959.0,13784.0,32175.0,50365.0,14457.0,0.0,4575.0,1820.0,15561806.0,338.601928


### Exclusion

Retail investors can define their exclusion criteria - for eg-  no exposure to Tobacco / Arms manufacturing / Oil

Other Criteria could be related Universe [for eg: Only US] / Disclosure Score [for eg: only top 50 percentile] 

In [61]:
# sorting for Performance bands -D, E - 

exclusion_performance_band = ['D','E','C']
list1 = data.loc[~data['Performance Band'].isin(exclusion)]

#Selecting companies in top 50 percentile of median score 

exclusion_disclosure_score = data["Disclosure Score"].median()
list1 = data.loc[data["Disclosure Score"]>exclusion_disclosure_score]

list1.columns

Index(['Company Name ', 'Account Number', 'Ticker Symbol ', 'ISIN ',
       'Disclosure Score', 'Performance Band', 'Permission', 'Response Status',
       'Scope 1 (metric tonnes CO2e)', 'Scope 2 (metric tonnes CO2e)',
       'Ticker', 'Reporting', 'Total Assets', 'Total Equity',
       'Total Liabilities ', 'Revenue', 'Gross Profit',
       'Research & Development', 'Operating Income (Loss)',
       'Net Income (Common)', 'Total Emissions', 'Emissions / Assets'],
      dtype='object')

In [62]:
num_of_names= 10


final_list= list1[['Company Name ','Ticker Symbol ', 
       'Disclosure Score', 'Performance Band', 'Scope 1 (metric tonnes CO2e)', 'Scope 2 (metric tonnes CO2e)',
       'Total Assets', 'Revenue', 'Gross Profit',
       'Net Income (Common)', 'Emissions / Assets']]


final_list.sort_values(by="Emissions / Assets").head(num_of_names)

,Company Name,Ticker Symbol,Disclosure Score,Performance Band,Scope 1 (metric tonnes CO2e),Scope 2 (metric tonnes CO2e),Total Assets,Revenue,Gross Profit,Net Income (Common),Emissions / Assets
167,"Allergan, Inc.",AGN US,91.0,B,49128.00,52049.00,128986.300,14570.600,12709.800,14695.000,0.784401
118,UnitedHealth Group Inc,UNH US,98.0,B,8693.78,104200.46,122810.000,184840.000,43386.000,7017.000,0.919259
40,"Adobe Systems, Inc.",ADBE US,97.0,A,2744.00,22995.00,12697.246,5854.430,5034.522,1168.782,2.027132
91,salesforce.com,CRM US,90.0,C,2349.52,29428.55,12762.920,6667.216,5012.668,-47.426,2.489875
143,Biogen Idec Inc.,BIIB US,92.0,B,46588.00,39307.00,22876.800,11448.800,9970.100,3702.800,3.754677
77,"Cisco Systems, Inc.",CSCO US,100.0,A,65832.00,628164.00,121652.000,49247.000,30960.000,10739.000,5.704764
39,Microsoft Corporation,MSFT US,96.0,A,48516.00,1207419.00,193468.000,91154.000,58374.000,20539.000,6.491694
129,Yahoo! Inc.,YHOO US,91.0,B,4002.04,333290.80,48083.079,5169.135,2450.241,-214.321,7.014793
38,Google Inc.,GOOG US,93.0,B,37187.00,1149988.00,167497.000,90272.000,55134.000,19478.000,7.087739
172,Johnson & Johnson,JNJ US,98.0,A-,329556.00,853700.00,141208.000,71890.000,50205.000,16540.000,8.379525
